In [4]:
import pyodbc  
import json

DataSource = "DSN=BaseSQL32"
ExportFile = "PyUnload"
ExportDataDir = "C:\\PyUnload2\\PyUnload\\Data"
Usuarios = ['PRG','MIGAP']
Tablas = ["GEstores",'Usuarios','Clientes','ProductosFotos']
RowsToExport = -1  # -1 para exportar todos

In [5]:
# Estrae la lista de TABLAS y VISTAS de un propietario especifico
 
def crearModeloDatos(DataSource, Usuarios, Tablas):

    modeloDatos = {}
    creator_id = ""
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    for user in Usuarios:

        # Se determina el ID del usuario propietario de las tablas
        sql_creador  = "SELECT * FROM SYSUSERS WHERE NAME = " + "'" + user + "'"
        cursor.execute(sql_creador)
        row = cursor.fetchone()
        creator_id=row.suid

        # Se extrae la lista de TABLAS de las cuales el usuario es propitario 
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = \'BASE\'"
        sql_tablas = sql_tablas + " AND CREATOR=" + str(creator_id)
        sql_tablas = sql_tablas + " ORDER BY TABLE_NAME "
        cursor.execute(sql_tablas)
        rows_table = cursor.fetchall()

        modeloDatos[user] = {}
        modeloDatos[user]["tables"] ={}

        for row_table in rows_table:

            #if  len(Tablas)!=0:
            if row_table.table_name.upper() in  [w.upper() for w in Tablas] :

                modeloDatos[user]["tables"][row_table.table_name] = {
                "table_owner" : user, 
                "table_name" : row_table.table_name,
                "table_id" : row_table.table_id,
                "table_count" : row_table.count }

                #print(row_table.table_name)

                # Extrae las columnas de cada tabla
                sql_columnas  = "SELECT * FROM SYSCOLUMNS WHERE TNAME = " + "'" + row_table.table_name + "'" + " ORDER BY COLNO"
                
                cursor.execute(sql_columnas)
                rows_columns = cursor.fetchall()
                modeloDatos[user]["tables"][row_table.table_name]["table_columns"] = []

                for row_column in rows_columns:
                    obj = {
                    "col_name" : row_column.cname,
                    "col_type" :row_column.coltype,
                    "col_null" :row_column.nulls,
                    "col_length" : row_column.length,
                    "col_syslength" : row_column.syslength,
                    "col_inPK": row_column.in_primary_key }
                    modeloDatos[user]["tables"][row_table.table_name]["table_columns"].append(obj)

                    #print("\t",row_column.cname)

                modeloDatos[user]["tables"][row_table.table_name]["table_indexes"] = []

                sql_indices  = "SELECT * FROM SYSINDEXES WHERE INDEXTYPE = 'Non-unique' AND TNAME =" + "'" + row_table.table_name + "' order by iname"   

                cursor.execute(sql_indices)
                rows_indices = cursor.fetchall()
                
                for rows_indice in rows_indices:
                    obj={}
                    obj["index_name"]=rows_indice.iname
                    colnames=rows_indice.colnames
                    obj["index_columns"]=colnames.split(',')
                    modeloDatos[user]["tables"][row_table.table_name]["table_indexes"] .append(obj)

                    #print("\t",rows_indice.tname)

                modeloDatos[user]["views"] = []

                # Se extrae la lista de VISTAS de las cuales el usuario es propitario 
                sql_vistas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = \'VIEW\'"
                sql_vistas = sql_vistas + " AND CREATOR=" + str(creator_id)
                sql_vistas = sql_vistas + " ORDER BY TABLE_NAME "
                
                cursor.execute(sql_vistas)
                rows_views = cursor.fetchall()

                for row_view in rows_views:
                    obj = { 
                    "view_name" : row_view.table_name,
                    "view_def" : row_view.view_def }
                    modeloDatos[user]["views"].append(obj)
        
    cursor.close()
    conexionDB.close()

    return modeloDatos

In [6]:
modeloDatos = crearModeloDatos(DataSource,Usuarios,Tablas)
file = open("ModeloDatos.json", "w" )
file.write(json.dumps(modeloDatos))
file.close()

NameError: name 'FiltroTablas' is not defined

In [ ]:
# Genera la sentencia DDL de CREATE TABLE de la tabla indicada en los parametros

def createTables(table_def ):

    DDL= ""

    DDL = DDL +  "IF EXISTS (SELECT * FROM sysobjects WHERE name = \'" + table_def["table_owner"] + "\' AND type = 'U') DROP TABLE " + table_def["table_owner"] + "." + table_def["table_name"] + ";\n"

    DDL = DDL + "CREATE TABLE " + "\"" + table_def["table_owner"] + "\".\"" + table_def["table_name"] +"\"\n" 
    DDL = DDL + "(" +"\n" 
    # Definicion de columnas
    for col in table_def["table_columns"]:
        DDL = DDL + "\t" + col["col_name"] + " " + col["col_type"]
        if col["col_type"] != "long varchar" and col["col_type"] != "integer" and col["col_type"] != "decimal" and  col["col_type"] != "date":
            DDL = DDL + "(" + str(col["col_length"]) + ")" 
        if col["col_type"] == "decimal":
            DDL = DDL +  "(" + str(col["col_length"]) +","+ str(col["col_syslength"]) + ")" 
        if col["col_null"] == "N":
            DDL = DDL + " NOT NULL" +",\n"
        else:
            DDL = DDL + ",\n"
    DDL =  DDL + ");\n"
    
    # Definicion de indices
    for ind in table_def["table_indexes"]:
        DDL = DDL + "CREATE INDEX " + "\"" + ind["index_name"] + "\" ON " + "\"" + table_def["table_owner"] + "\".\"" + table_def["table_name"] +"\"\n" 
        DDL = DDL + "(" +"\n" 
        for col in ind["index_columns"]:
            DDL = DDL + "\t"  + col + ",\n"
        DDL = DDL + ");\n"


    return DDL

In [ ]:
for usuario, objetos in modeloDatos.items():
    file = open("PyUnload-" + usuario + ".sql", "w" )
    for tabla, tblDef in modeloDatos[usuario]["tables"].items():
        DDL= createTables(tblDef)
        file.write(DDL)
    file.close()


In [ ]:
def exportData(DataSource, tblDef, RowsToExport, ExportDataDir ):

    def rowConversion(row, columns_types ):
        
        str_row = ""
        ind = 0

        try: 
            for data in row:
                if type(data) == 'NoneType':
                    data = ""
                if columns_types[ind]=='char' :
                    str_row = str_row + "\'" + data + "\'"
                if columns_types[ind]=='decimal' :
                    str_row = str_row + str(float(data))
                if columns_types[ind]=='date' :
                    d = str(data)
                    r = "\'" + d.replace("datetime.date(","").replace(")","").replace(" ","").replace(",","-") + "\'" 
                    str_row = str_row + r
                else:
                    str_row = str_row + str(data)
                    
                if ind<len(row)-1:
                    str_row = str_row + ","
            
                ind = ind + 1
        except:
            #print(type(data))
            #print(data)
            pass
        
        return str_row  

    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    if tblDef["table_count"]==0:
        return "", False

    i = 0 
    columns_names = ""
    columns_types = []
    for n in tblDef["table_columns"]:
        columns_names = columns_names + n["col_name"]
        columns_types.append( n["col_type"])
        if i < len(tblDef["table_columns"])-1:
            columns_names = columns_names + ","
            i = i + 1

    #print(columns_names)
    #print(columns_types)
    
    if RowsToExport!=-1:
        sql  = "select top " + str(RowsToExport) + " " + columns_names + " from " + tblDef["table_owner"] + "." + tblDef["table_name"] 
    else:
        sql  = "select " + columns_names + " from " + tblDef["table_owner"] + "." +  tblDef["table_name"]

    #print(sql)
    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  ExportDataDir + "\\" + str(tblDef["table_id"]) + ".dat"

    #print(len(rows))
    
    file = open(archivo_dat, "w")
    for row in rows:
        file.write(rowConversion(row,columns_types) +"\n")
    
    file.close()

    return archivo_dat , True

In [ ]:
for usuario, objetos in modeloDatos.items():
    for tabla, tblDef in modeloDatos[usuario]["tables"].items():
        archivo_dat= exportData(DataSource, tblDef, RowsToExport, ExportDataDir)
        #print(archivo_dat)
